# Libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
import torch.nn.functional as F
import numpy as np

from sklearn.model_selection import train_test_split

batch_size = 64

# Load Data

# Data Preprocessing

In [ ]:
# Data Loader (X and y)
X = [] # fill with data
y = [] # fill with data

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train)
X_val_tensor = torch.tensor(X_val)
y_train_tensor = torch.tensor(y_train)
y_val_tensor = torch.tensor(y_val)

training_dataset = TensorDataset(X_train_tensor, y_train_tensor)
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = TensorDataset(X_val_tensor, y_val_tensor)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

# RNN

In [ ]:
# RNN Variables
seq_len = 256
embed_size = 512 # larger embed size may require larger dropout
dropout = 0.2
lr = 1e-4
epochs = 20

In [ ]:
class Music_RNN(nn.Module):
    def __init__(self, input_size, embed_size, dropout):
        super(Music_RNN, self).__init__()
        self.RNN = nn.RNN(input_size, embed_size, batch_first=True, dropout=dropout)
        self.ff = nn.Linear(embed_size, 1) # predicting next input autoregressively

    def forward(self, x):
        # x should be (batch_size, seq_len)
        x = self.RNN(x)
        x_pred = self.ff(x)

        return x_pred # (batch_size, 1)

In [ ]:
model_rnn = Music_RNN(seq_len, embed_size, dropout)

In [ ]:
# adapt loss and optimizer as needed
criterion = nn.MSELoss()
optimizer = Adam(model_rnn.parameters(), lr=lr)

# train
for epoch in range(epochs):
    # Set to train
    model_rnn.train()
    # keep cumalitive losses
    total_losses = 0.0

    for batch in enumerate(training_dataloader):
        inputs, targets = batch
        optimizer.zero_grad()

        preds = model_rnn.forward(inputs) # make sure dimensions line up

        loss = criterion(preds, targets)
        loss.backward()

        optimizer.step()

        total_losses += loss.item()

    print(f"Epoch: {epoch}, Loss: {total_losses / len(training_dataloader)}")


In [ ]:
# Validation

model_rnn.eval()

total_losses = 0.0

# validation
with torch.no_grad():
    for batch in validation_dataloader:
        inputs, targets = batch

        preds = model_rnn.forward(inputs) # make sure dimensions line up

        loss = criterion(preds, targets)
        total_losses += loss.item()

    print(f"Loss: {total_losses / len(validation_dataloader)}")

# LSTM

# Transformer